## 基于MindSpore框架的DenseNet案例实现

### 1 模型简介

DenseNet模型于2017年在论文《Densely Connected Convolutional Networks》中被提出。DenseNet通过特征在channel上的连接来实现特征重用（feature reuse）。这些特点让DenseNet在参数和计算成本更少的情形下实现比ResNet更优的性能，DenseNet也因此斩获CVPR 2017的最佳论文奖。本篇文章首先介绍DenseNet的原理以及网路架构，然后讲解DenseNet在MindSpore上的实现。

#### 1.1 模型结构
相比ResNet，DenseNet提出了一个更激进的密集连接机制：即互相连接所有的层，具体来说就是每个层都会接受其前面所有层作为其额外的输入。图1为ResNet网络的连接机制，作为对比，图1为DenseNet的密集连接机制。可以看到，ResNet是每个层与前面的某层（一般是2~3层）短路连接在一起，连接方式是通过元素级相加。而在DenseNet中，每个层都会与前面所有层在通道维度上连接在一起，并作为下一层的输入。对于一个L层的网络，DenseNet共包含$ \frac{L(L+1)}{2}$个连接，相比ResNet，这是一种密集连接。而且DenseNet是直接连接来自不同层的特征图，这可以实现特征重用，提升效率，这一特点是DenseNet与ResNet最主要的区别。

<center>
    <img src="https://s1.ax1x.com/2022/09/24/xAeebR.jpg" alt="image-2022092401" style="zoom:75%;" />
    <br>
    <div style="color:orange;
    display: inline-block;
    color: #999;
    padding: 2px;">图1 一个5层的密集块，增长率为k=4。每一层都将所有前面的特征图作为输入。</div>
</center>

传统的网络在L层的输出为：
$$
x_{l}=H_{l}(x_{l−1})
$$

而对于ResNet，增加了来自上一层输入的identity函数：

$$
x_{l} = H_{l}(x_{l−1})+x_{l-1}
$$

在DenseNet中，会连接前面所有层作为输入：
$$
x_{l}=H_{l}([x_{0},x_{1},...,x_{l−1}])
$$

其中，上面的$H_{l}( ⋅ )$代表是非线性转化函数（non-liear transformation），它是一个组合操作，其可能包括一系列的BN(Batch Normalization)，ReLU，Pooling及Conv操作。注意这里l层与 $l−1$层之间可能实际上包含多个卷积层。

CNN网络一般要经过Pooling或者stride>1的Conv来降低特征图的大小，而DenseNet的密集连接方式需要特征图大小保持一致。为了解决这个问题，DenseNet网络中使用DenseBlock+Transition的结构，其中DenseBlock是包含很多层的模块，每个层的特征图大小相同，层与层之间采用密集连接方式。而Transition模块是连接两个相邻的DenseBlock，并且通过Pooling使特征图大小降低。图2给出了具有三个密集块的深度 DenseNet，各个DenseBlock之间通过Transition连接在一起。

<center>
    <img src="https://s1.ax1x.com/2022/09/24/xAu3xU.jpg" alt="image-2022092402" style="zoom:75%;" />
    <br>
    <div style="color:orange;
    display: inline-block;
    color: #999;
    padding: 2px;">图2 具有三个密集块的深度 DenseNet。 两个相邻块之间的层称为过渡层，并通过卷积和池化改变特征图大小。</div>
</center>


#### 1.2 DenseNet优点

a) 相比ResNet拥有更少的参数数量；

b) 旁路加强了特征的重用；

c) 网络更易于训练,并具有一定的正则效果；

d) 缓解了gradient vanishing和model degradation的问题。


## 2. 案例实现

### 2.1 数据集下载与准备
+ Mini-ImageNet：https://drive.google.com/drive/folders/17a09kkqVivZQFggCw9I_YboJ23tcexNM

+ 下载的数据集根目录dataset/下有train、val、test文件夹
+ 但由于该数据集用于元学习和小样本学习，数据集的划分不是从每个类别中进行采样的，所以需要进一步对下载的数据集重新进行train、val、test的划分，使train、val、test都包含全部的分类

In [2]:
import os, random, shutil

# 先把上述链接中下载好的val、test目录下的所有文件夹都剪切至train目录，完成合并
# 如下代码是使train、val、test都含有全部100个分类，并针对每个分类按照6:2:2的比例划分

def moveFile(fileDir, tarDir, rate):
    pathDir = os.listdir(fileDir)  # 取图片的原始路径
    filenumber = len(pathDir)
    if filenumber > 360: # 重新划分好的train目录下每个分类的图片数量=360，此行是为了防止重复进行划分
        rate = rate  # 自定义抽取图片的比例，比方说100张抽10张，那就是0.1
        picknumber = int(filenumber * rate)  # 按照rate比例从文件夹中取一定数量图片
        sample = random.sample(pathDir, picknumber)  # 随机选取picknumber数量的样本图片
        for name in sample:
            shutil.move(fileDir + '/' + name, tarDir + '/' + name)

data_dir = "dataset/train"

for classes_num in os.listdir(data_dir):
    src_dir = os.path.join("./dataset/train/",classes_num)
    dst_dir_val = os.path.join("./dataset/val/", classes_num)
    dst_dir_test = os.path.join("./dataset/test/", classes_num)
    
    if not os.path.exists(dst_dir_val):
        os.mkdir(dst_dir_val)
    
    if not os.path.exists(dst_dir_test):
        os.mkdir(dst_dir_test)
    
    moveFile(src_dir, dst_dir_val, 0.2)
    moveFile(src_dir, dst_dir_test, 0.25)

+ 重新划分后，train、val、test每个文件夹下目录结构如下：
```
.
└── train/val/test 
     ├── class1
     │    ├── 000000000001.jpg
     │    ├── 000000000002.jpg
     │    ├── ...
     ├── class2
     │    ├── 000000000001.jpg
     │    ├── 000000000002.jpg
     │    ├── ...
     ├── class3
     │    ├── 000000000001.jpg
     │    ├── 000000000002.jpg
     │    ├── ...
     ├── class100
          ├── 000000000001.jpg
          ├── 000000000002.jpg
          ├── ...
```

### 2.2 数据集创建
在创建数据集时使用了mindspore框架提供的ImageFolderDataset，该方法可以从从树状结构的文件目录中读取图片构建源数据集，也就是在2.1中处理好的train/val/test目录，且这3个目录下都包含100个分类目录，同一个分类目录的所有图片都会被分配相同的label，由此即可返回生成好的两列数据集：\[image, label\]。此外数据集创建时还进行了包括数据增强，归一化等transfrom定义，之后通过mindspore的map方法来分别完成样本和label的transform映射。

In [13]:
import os
import math
import numpy as np
from PIL import Image, ImageFile
from mindspore import dtype as mstype
import mindspore.dataset as de
import mindspore.dataset.vision.c_transforms as vision_C
import mindspore.dataset.transforms.c_transforms as normal_C

ImageFile.LOAD_TRUNCATED_IMAGES = True

def classification_dataset(data_dir, image_size, per_batch_size, max_epoch, rank,group_size,
                           mode='train', input_mode='folder', root='', num_parallel_workers=None,
                           shuffle=None, sampler=None, class_indexing=None, drop_remainder=True,
                           transform=None, target_transform=None):
   

    mean = [0.485 * 255, 0.456 * 255, 0.406 * 255]
    std = [0.229 * 255, 0.224 * 255, 0.225 * 255]

    if transform is None:
        if mode == 'train':
            transform_img = [
                vision_C.RandomCropDecodeResize(image_size, scale=(0.08, 1.0), ratio=(0.75, 1.333)),
                vision_C.RandomHorizontalFlip(prob=0.5),
                vision_C.RandomColorAdjust(brightness=0.4, contrast=0.4, saturation=0.4),
                vision_C.Normalize(mean=mean, std=std),
                vision_C.HWC2CHW()]
        else:
            transform_img = [
                vision_C.Decode(),
                vision_C.Resize((256, 256)),
                vision_C.CenterCrop(image_size),
                vision_C.Normalize(mean=mean, std=std),
                vision_C.HWC2CHW()]
    else:
        transform_img = transform

    if target_transform is None:
        transform_label = [normal_C.TypeCast(mstype.int32)]
    else:
        transform_label = target_transform


    de_dataset = de.ImageFolderDataset(data_dir, num_parallel_workers=num_parallel_workers,
                                       shuffle=shuffle, sampler=sampler, class_indexing=class_indexing,
                                       num_shards=group_size, shard_id=rank)

    de_dataset = de_dataset.map(input_columns="image", num_parallel_workers=8, operations=transform_img)
    de_dataset = de_dataset.map(input_columns="label", num_parallel_workers=8, operations=transform_label)

    columns_to_project = ["image", "label"]
    de_dataset = de_dataset.project(columns=columns_to_project)

    de_dataset = de_dataset.batch(per_batch_size, drop_remainder=drop_remainder)
    de_dataset = de_dataset.repeat(1)

    return de_dataset

In [14]:
data_dir = "./dataset/train"
train_dataset = classification_dataset(data_dir, image_size=[224, 224],per_batch_size=4, max_epoch=20, rank=0, group_size=1)
for image,label in train_dataset:
    print(image.shape, label)
    


[WARNING] ME(95841:8663750144,MainProcess):2022-09-25-17:48:11.629.925 [mindspore/dataset/core/validator_helpers.py:804] 'RandomCropDecodeResize' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'RandomCropDecodeResize' from mindspore.dataset.vision instead.
[WARNING] ME(95841:8663750144,MainProcess):2022-09-25-17:48:11.634.068 [mindspore/dataset/core/validator_helpers.py:804] 'RandomHorizontalFlip' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'RandomHorizontalFlip' from mindspore.dataset.vision instead.
[WARNING] ME(95841:8663750144,MainProcess):2022-09-25-17:48:11.635.752 [mindspore/dataset/core/validator_helpers.py:804] 'RandomColorAdjust' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'RandomColorAdjust' from mindspore.dataset.vision instead.
[WARNING] ME(95841:86637501

(4, 3, 224, 224) [91 33 14 90]
(4, 3, 224, 224) [21 59 91 57]
(4, 3, 224, 224) [21 86 88 60]
(4, 3, 224, 224) [99 23 55 63]
(4, 3, 224, 224) [75 88 68 54]
(4, 3, 224, 224) [47 72 94 93]
(4, 3, 224, 224) [10  7 83 90]
(4, 3, 224, 224) [ 1 47 25 95]
(4, 3, 224, 224) [45 94 30 95]
(4, 3, 224, 224) [54  5 26 98]
(4, 3, 224, 224) [82 83 78  0]
(4, 3, 224, 224) [77 34 32 38]
(4, 3, 224, 224) [28  3 27  3]
(4, 3, 224, 224) [54 48 46 42]
(4, 3, 224, 224) [57 30 77 47]
(4, 3, 224, 224) [26 95 30 67]
(4, 3, 224, 224) [11  0 95 94]
(4, 3, 224, 224) [66 14 26 38]
(4, 3, 224, 224) [28 92 81 11]
(4, 3, 224, 224) [85 93 63 17]
(4, 3, 224, 224) [98  3 35 67]
(4, 3, 224, 224) [27 36 11 37]
(4, 3, 224, 224) [33 40 46  1]
(4, 3, 224, 224) [31 63 45 73]
(4, 3, 224, 224) [44 70 55 14]
(4, 3, 224, 224) [82  6 80 62]
(4, 3, 224, 224) [41 45 20 42]
(4, 3, 224, 224) [38 74  0 64]
(4, 3, 224, 224) [51 63 18  5]
(4, 3, 224, 224) [61 96  8 46]
(4, 3, 224, 224) [92 22 60  6]
(4, 3, 224, 224) [22 70 10  5]
(4, 3, 2

(4, 3, 224, 224) [28 32 58 64]
(4, 3, 224, 224) [37 26 56  9]
(4, 3, 224, 224) [47 74 93 86]
(4, 3, 224, 224) [96 40 40 58]
(4, 3, 224, 224) [16 54 93 40]
(4, 3, 224, 224) [ 2 73 89 84]
(4, 3, 224, 224) [10 97 14 16]
(4, 3, 224, 224) [57 95  1 78]
(4, 3, 224, 224) [35 22 26 85]
(4, 3, 224, 224) [69 59 66 94]
(4, 3, 224, 224) [28  0 85 37]
(4, 3, 224, 224) [73 13 96 28]
(4, 3, 224, 224) [42 99 13 68]
(4, 3, 224, 224) [97  2 52  9]
(4, 3, 224, 224) [98 95 49 24]
(4, 3, 224, 224) [91 10 23 50]
(4, 3, 224, 224) [14 16  4 25]
(4, 3, 224, 224) [28 62 33 40]
(4, 3, 224, 224) [79 14 61 37]
(4, 3, 224, 224) [11 72 67 42]
(4, 3, 224, 224) [ 4 45 38 79]
(4, 3, 224, 224) [69 47 77 54]
(4, 3, 224, 224) [99 18 11 69]
(4, 3, 224, 224) [12 19 25 78]
(4, 3, 224, 224) [19 33 66 61]
(4, 3, 224, 224) [96 22  1 94]
(4, 3, 224, 224) [85 33  2 10]
(4, 3, 224, 224) [47 98 45 15]
(4, 3, 224, 224) [96 24  2 44]
(4, 3, 224, 224) [53 84 96 74]
(4, 3, 224, 224) [61 61 83 12]
(4, 3, 224, 224) [30 70 57 38]
(4, 3, 2

(4, 3, 224, 224) [65 26 38  9]
(4, 3, 224, 224) [37 78 46 75]
(4, 3, 224, 224) [44 52  9 59]
(4, 3, 224, 224) [83 30 28 56]
(4, 3, 224, 224) [95 13 48 40]
(4, 3, 224, 224) [27 68 92 25]
(4, 3, 224, 224) [24 67 63 64]
(4, 3, 224, 224) [88 91 76 13]
(4, 3, 224, 224) [64 63 52 56]
(4, 3, 224, 224) [ 1  9 33 69]
(4, 3, 224, 224) [71 41  8 52]
(4, 3, 224, 224) [59 27 46 35]
(4, 3, 224, 224) [12 22 96 45]
(4, 3, 224, 224) [97 78 60 63]
(4, 3, 224, 224) [83 91 95 40]
(4, 3, 224, 224) [27 76 62 31]
(4, 3, 224, 224) [38 83 14 80]
(4, 3, 224, 224) [32 29 41 60]
(4, 3, 224, 224) [86 62 42 65]
(4, 3, 224, 224) [59 55 88 67]
(4, 3, 224, 224) [43 61 35 80]
(4, 3, 224, 224) [65 24 92 55]
(4, 3, 224, 224) [98 22 53 39]
(4, 3, 224, 224) [88 40 66 96]
(4, 3, 224, 224) [37 30 99 85]
(4, 3, 224, 224) [45 31  0 11]
(4, 3, 224, 224) [29 71 93 90]
(4, 3, 224, 224) [41 73  2 66]
(4, 3, 224, 224) [60  0  2 62]
(4, 3, 224, 224) [88 35 51 67]
(4, 3, 224, 224) [16 14 95  6]
(4, 3, 224, 224) [11  4 58 53]
(4, 3, 2

(4, 3, 224, 224) [31 81 21 19]
(4, 3, 224, 224) [69 51 19  4]
(4, 3, 224, 224) [99 24  8 44]
(4, 3, 224, 224) [43 69 99 97]
(4, 3, 224, 224) [99 73 59 41]
(4, 3, 224, 224) [ 9 19 38 36]
(4, 3, 224, 224) [94  3 82 19]
(4, 3, 224, 224) [30 26 87 59]
(4, 3, 224, 224) [26 23 97 80]
(4, 3, 224, 224) [32 79 80  6]
(4, 3, 224, 224) [ 9 11 56 50]
(4, 3, 224, 224) [99 48 38 33]
(4, 3, 224, 224) [50 14 42 42]
(4, 3, 224, 224) [33 40  2 74]
(4, 3, 224, 224) [98 91 66 64]
(4, 3, 224, 224) [57 18 97 50]
(4, 3, 224, 224) [68 94 86 13]
(4, 3, 224, 224) [91 32 45 19]
(4, 3, 224, 224) [79 10 26 66]
(4, 3, 224, 224) [15 45 38 59]
(4, 3, 224, 224) [39 53 94 58]
(4, 3, 224, 224) [26 85 91 37]
(4, 3, 224, 224) [17 16 62 46]
(4, 3, 224, 224) [40 93 60 34]
(4, 3, 224, 224) [37 82 66 72]
(4, 3, 224, 224) [43 25 94 58]
(4, 3, 224, 224) [33 68 56 94]
(4, 3, 224, 224) [38 58  8 22]
(4, 3, 224, 224) [28  2  5 40]
(4, 3, 224, 224) [94 41 37 62]
(4, 3, 224, 224) [66 68 35 97]
(4, 3, 224, 224) [56 47 73  9]
(4, 3, 2

...

(4, 3, 224, 224) [30 54 65 80]
(4, 3, 224, 224) [58 50 40 84]
(4, 3, 224, 224) [ 4 85 53 68]
(4, 3, 224, 224) [21 70 14  6]
(4, 3, 224, 224) [73 96 12 26]
(4, 3, 224, 224) [67  1 16 82]
(4, 3, 224, 224) [72  4 97 33]
(4, 3, 224, 224) [ 7 93 51 40]
(4, 3, 224, 224) [50 97 20 34]
(4, 3, 224, 224) [16  2 54 86]
(4, 3, 224, 224) [20 95 71  8]
(4, 3, 224, 224) [33 84 29 90]
(4, 3, 224, 224) [20 21 16 61]
(4, 3, 224, 224) [73 49 80 23]
(4, 3, 224, 224) [97 43 59 37]
(4, 3, 224, 224) [ 9 12 74 55]
(4, 3, 224, 224) [59 91  6  6]
(4, 3, 224, 224) [83 68 34 85]
(4, 3, 224, 224) [89 25 57 23]
(4, 3, 224, 224) [84 16 59 25]
(4, 3, 224, 224) [86 11  8 14]
(4, 3, 224, 224) [85 37 64 84]
(4, 3, 224, 224) [21 69 95 60]
(4, 3, 224, 224) [36 37 34 78]
(4, 3, 224, 224) [25 89 63 72]
(4, 3, 224, 224) [ 0  1 76 79]
(4, 3, 224, 224) [44 82 50 47]
(4, 3, 224, 224) [20  9 90 37]
(4, 3, 224, 224) [39 28 91 73]
(4, 3, 224, 224) [37 84  9 17]
(4, 3, 224, 224) [91  2 73 38]
(4, 3, 224, 224) [82 45 64 27]
(4, 3, 2

(4, 3, 224, 224) [48 85 69 48]
(4, 3, 224, 224) [46 29 29 92]
(4, 3, 224, 224) [ 1 16 62 73]
(4, 3, 224, 224) [ 0 64 16 31]
(4, 3, 224, 224) [41 55 54 83]
(4, 3, 224, 224) [66 71  6 38]
(4, 3, 224, 224) [99 33 86 94]
(4, 3, 224, 224) [46 61 11 82]
(4, 3, 224, 224) [70 47 52 45]
(4, 3, 224, 224) [88 91 27 51]
(4, 3, 224, 224) [52 84 94  5]
(4, 3, 224, 224) [81 68 92 97]
(4, 3, 224, 224) [91 68 38  6]
(4, 3, 224, 224) [91 35 99 85]
(4, 3, 224, 224) [63 55  5  4]
(4, 3, 224, 224) [85 98 11 58]
(4, 3, 224, 224) [56 43 85 37]
(4, 3, 224, 224) [ 6 90 11 20]
(4, 3, 224, 224) [81 37 37 78]
(4, 3, 224, 224) [70 98  4 84]
(4, 3, 224, 224) [56 50  3 16]
(4, 3, 224, 224) [88  2 62 27]
(4, 3, 224, 224) [20 87  0 21]
(4, 3, 224, 224) [ 8 10 30 87]
(4, 3, 224, 224) [84 59 18  7]
(4, 3, 224, 224) [45 31 89 65]
(4, 3, 224, 224) [35 32 32 36]
(4, 3, 224, 224) [ 3 53 62 44]
(4, 3, 224, 224) [50 29 64 57]
(4, 3, 224, 224) [ 7 18  6 49]
(4, 3, 224, 224) [13 55 44  4]
(4, 3, 224, 224) [ 2 37 80  0]
(4, 3, 2

(4, 3, 224, 224) [60 55 15 96]
(4, 3, 224, 224) [97 25 42 94]
(4, 3, 224, 224) [38 72 39 26]
(4, 3, 224, 224) [72  1 59 65]
(4, 3, 224, 224) [27 73 99 79]
(4, 3, 224, 224) [94 20 71 88]
(4, 3, 224, 224) [18 72 35 44]
(4, 3, 224, 224) [68 98 71 54]
(4, 3, 224, 224) [67 79 71  4]
(4, 3, 224, 224) [75 67 10  2]
(4, 3, 224, 224) [ 8 30 53 70]
(4, 3, 224, 224) [81 22 23  2]
(4, 3, 224, 224) [13 52 94 90]
(4, 3, 224, 224) [82 15 90 31]
(4, 3, 224, 224) [ 0  3 12 38]
(4, 3, 224, 224) [35 66 65 35]
(4, 3, 224, 224) [71 20 80 56]
(4, 3, 224, 224) [14 23 18 61]
(4, 3, 224, 224) [ 1 19 13  9]
(4, 3, 224, 224) [55 88 48 21]
(4, 3, 224, 224) [54 16 65 78]
(4, 3, 224, 224) [80 42 16  3]
(4, 3, 224, 224) [18 96 63 86]
(4, 3, 224, 224) [61 82 17  8]
(4, 3, 224, 224) [66  6 94 38]
(4, 3, 224, 224) [23 63 78 15]
(4, 3, 224, 224) [97 17 51 52]
(4, 3, 224, 224) [29  4 84  5]
(4, 3, 224, 224) [98 34 90 11]
(4, 3, 224, 224) [33 71  2 74]
(4, 3, 224, 224) [ 6 23 75 94]
(4, 3, 224, 224) [15 67 46 82]
(4, 3, 2

(4, 3, 224, 224) [64 48 65 16]
(4, 3, 224, 224) [26 37 35 93]
(4, 3, 224, 224) [10 84 98 86]
(4, 3, 224, 224) [37 67 96 12]
(4, 3, 224, 224) [34 63 85 10]
(4, 3, 224, 224) [82  5 76  8]
(4, 3, 224, 224) [54 45  3 81]
(4, 3, 224, 224) [56 94  6 26]
(4, 3, 224, 224) [96 57 79 12]
(4, 3, 224, 224) [70 75 45 12]
(4, 3, 224, 224) [32  6 70 90]
(4, 3, 224, 224) [30 71 94 67]
(4, 3, 224, 224) [31 25 97 61]
(4, 3, 224, 224) [ 5 53 85 10]
(4, 3, 224, 224) [ 4 81 40 13]
(4, 3, 224, 224) [68 32  7 19]
(4, 3, 224, 224) [62 29 31 49]
(4, 3, 224, 224) [98  9 60  6]
(4, 3, 224, 224) [17 90 13 72]
(4, 3, 224, 224) [95 97 84 13]
(4, 3, 224, 224) [37 47  8 60]
(4, 3, 224, 224) [96 58 36 33]
(4, 3, 224, 224) [78 51 26 91]
(4, 3, 224, 224) [ 4 57  2 67]
(4, 3, 224, 224) [43 90  6 23]
(4, 3, 224, 224) [40 92 98 61]
(4, 3, 224, 224) [82 72 86 42]
(4, 3, 224, 224) [15 27 90 82]
(4, 3, 224, 224) [63 30 18 62]
(4, 3, 224, 224) [ 0 60 64 74]
(4, 3, 224, 224) [81 95 62 28]
(4, 3, 224, 224) [91 50 98 28]
(4, 3, 2

(4, 3, 224, 224) [15 55 49 99]
(4, 3, 224, 224) [52 32 71 67]
(4, 3, 224, 224) [80 88 24 92]
(4, 3, 224, 224) [69 61 49 67]
(4, 3, 224, 224) [43  0 75 58]
(4, 3, 224, 224) [ 5 44 66 53]
(4, 3, 224, 224) [94 10 26 31]
(4, 3, 224, 224) [35 33 30 49]
(4, 3, 224, 224) [77 94 37 84]
(4, 3, 224, 224) [83 89 14 81]
(4, 3, 224, 224) [53 94 92  8]
(4, 3, 224, 224) [72 78 85  3]
(4, 3, 224, 224) [68 44 69 43]
(4, 3, 224, 224) [52 35 65 67]
(4, 3, 224, 224) [83 88 40  7]
(4, 3, 224, 224) [ 9 63 16 45]
(4, 3, 224, 224) [26 74 85 65]
(4, 3, 224, 224) [ 7 52 14 53]
(4, 3, 224, 224) [13 90 14  3]
(4, 3, 224, 224) [71 20  8 70]
(4, 3, 224, 224) [89 37 14 50]
(4, 3, 224, 224) [28 88 54 51]
(4, 3, 224, 224) [63 48 51 65]
(4, 3, 224, 224) [94 97  0 91]
(4, 3, 224, 224) [11 48 99  0]
(4, 3, 224, 224) [92 36 92 99]
(4, 3, 224, 224) [60 33 98 30]
(4, 3, 224, 224) [24 74 85 14]
(4, 3, 224, 224) [52  7 81 33]
(4, 3, 224, 224) [82 36 13 12]
(4, 3, 224, 224) [39 85 11 11]
(4, 3, 224, 224) [24 33 16 59]
(4, 3, 2

### 2.3 模型构建

In [16]:
import math
from functools import reduce
from collections import OrderedDict
import mindspore.nn as nn
import mindspore as ms
from mindspore import Tensor
from mindspore.ops import operations as P
from mindspore.common import initializer as init
#from utils.network_utils import default_recurisive_init, KaimingNormal


__all__ = ["DenseNet121", "DenseNet100"]

+ 定义Kaiming初始化函数

本案例在构建Densenet121网络时，需要计算的Kaiming初始化函数，根据Kaiming Normal计算公式：$ \frac{gain}{\sqrt{fan}}$，具体流程首先时定义了KamingInit类，并通过_calculate_gain函数计算公式分子gain，再通过定义继承了KamingInit类的KamingNormal类，完成分母fan的计算，完成针对数组的Kaiming初始化函数。

In [ ]:
def _calculate_gain(nonlinearity, param=None):
    linear_fns = ['linear', 'conv1d', 'conv2d', 'conv3d', 'conv_transpose1d', 'conv_transpose2d', 'conv_transpose3d']
    if nonlinearity in linear_fns or nonlinearity == 'sigmoid':
        return 1
    if nonlinearity == 'tanh':
        return 5.0 / 3
    if nonlinearity == 'relu':
        return math.sqrt(2.0)
    if nonlinearity == 'leaky_relu':
        if param is None:
            negative_slope = 0.01
        elif not isinstance(param, bool) and isinstance(param, int) or isinstance(param, float):
            negative_slope = param
        else:
            raise ValueError("negative_slope {} not a valid number".format(param))
        return math.sqrt(2.0 / (1 + negative_slope ** 2))

    raise ValueError("Unsupported nonlinearity {}".format(nonlinearity))
    
    
def _assignment(arr, num):
    if arr.shape == ():
        arr = arr.reshape((1))
        arr[:] = num
        arr = arr.reshape(())
    else:
        if isinstance(num, np.ndarray):
            arr[:] = num[:]
        else:
            arr[:] = num
    return arr


def _calculate_in_and_out(arr):
    dim = len(arr.shape)
    if dim < 2:
        raise ValueError("If initialize data with xavier uniform, the dimension of data must greater than 1.")

    n_in = arr.shape[1]
    n_out = arr.shape[0]

    if dim > 2:
        counter = reduce(lambda x, y: x * y, arr.shape[2:])
        n_in *= counter
        n_out *= counter
    return n_in, n_out


class KaimingInit(init.Initializer):
    def __init__(self, a=0, mode='fan_in', nonlinearity='leaky_relu'):
        super(KaimingInit, self).__init__()
        self.mode = mode
        self.gain = _calculate_gain(nonlinearity, a)

    def _initialize(self, arr):
        pass
    

class KaimingNormal(KaimingInit):
    def _initialize(self, arr):
        fan = _select_fan(arr, self.mode)
        std = self.gain / math.sqrt(fan)
        data = np.random.normal(0, std, arr.shape)

        _assignment(arr, data)


def default_recurisive_init(custom_cell):
    for _, cell in custom_cell.cells_and_names():
        if isinstance(cell, nn.Conv2d):
            cell.weight.set_data(init.initializer(KaimingUniform(a=math.sqrt(5)), cell.weight.shape, cell.weight.dtype))
            if cell.bias is not None:
                fan_in, _ = _calculate_in_and_out(cell.weight.asnumpy())
                bound = 1 / math.sqrt(fan_in)
                cell.bias.set_data(Tensor(np.random.uniform(-bound, bound, cell.bias.shape), cell.bias.dtype))
        elif isinstance(cell, nn.Dense):
            cell.weight.set_data(init.initializer(KaimingUniform(a=math.sqrt(5)), cell.weight.shape, cell.weight.dtype))
            if cell.bias is not None:
                fan_in, _ = _calculate_in_and_out(cell.weight.asnumpy())
                bound = 1 / math.sqrt(fan_in)
                cell.bias.set_data(Tensor(np.random.uniform(-bound, bound, cell.bias.shape), cell.bias.dtype))
        elif isinstance(cell, (nn.BatchNorm2d, nn.BatchNorm1d)):
            pass


+ 构建DenseBlock

In [17]:
class GlobalAvgPooling(nn.Cell):
    def __init__(self):
        super(GlobalAvgPooling, self).__init__()
        self.mean = P.ReduceMean(True)
        self.shape = P.Shape()
        self.reshape = P.Reshape()

    def construct(self, x):
        x = self.mean(x, (2, 3))
        b, c, _, _ = self.shape(x)
        x = self.reshape(x, (b, c))
        return x

class CommonHead(nn.Cell):
    def __init__(self, num_classes, out_channels):
        super(CommonHead, self).__init__()
        self.avgpool = GlobalAvgPooling()
        self.fc = nn.Dense(out_channels, num_classes, has_bias=True)

    def construct(self, x):
        x = self.avgpool(x)
        x = self.fc(x)
        return x

def conv7x7(in_channels, out_channels, stride=1, padding=3, has_bias=False):
    return nn.Conv2d(in_channels, out_channels, kernel_size=7, stride=stride, has_bias=has_bias,
                     padding=padding, pad_mode="pad")


def conv3x3(in_channels, out_channels, stride=1, padding=1, has_bias=False):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, has_bias=has_bias,
                     padding=padding, pad_mode="pad")


def conv1x1(in_channels, out_channels, stride=1, padding=0, has_bias=False):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, has_bias=has_bias,
                     padding=padding, pad_mode="pad")


class _DenseLayer(nn.Cell):
    """
    the dense layer, include 2 conv layer
    """
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.norm1 = nn.BatchNorm2d(num_input_features)
        self.relu1 = nn.ReLU()
        self.conv1 = conv1x1(num_input_features, bn_size*growth_rate)

        self.norm2 = nn.BatchNorm2d(bn_size*growth_rate)
        self.relu2 = nn.ReLU()
        self.conv2 = conv3x3(bn_size*growth_rate, growth_rate)

        # nn.Dropout in MindSpore use keep_prob, diff from Pytorch
        self.keep_prob = 1.0 - drop_rate
        self.dropout = nn.Dropout(keep_prob=self.keep_prob)

    def construct(self, features):
        bottleneck = self.conv1(self.relu1(self.norm1(features)))
        new_features = self.conv2(self.relu2(self.norm2(bottleneck)))
        if self.keep_prob < 1:
            new_features = self.dropout(new_features)
        return new_features

class _DenseBlock(nn.Cell):
    """
    the dense block
    """
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        self.cell_list = nn.CellList()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate
            )
            self.cell_list.append(layer)

        self.concate = P.Concat(axis=1)

    def construct(self, init_features):
        features = init_features
        for layer in self.cell_list:
            new_features = layer(features)
            features = self.concate((features, new_features))
        return features

+ 构建Transition层

In [18]:
class _Transition(nn.Cell):
    """
    the transition layer
    """
    def __init__(self, num_input_features, num_output_features, avgpool=False):
        super(_Transition, self).__init__()
        if avgpool:
            poollayer = nn.AvgPool2d(kernel_size=2, stride=2)
        else:
            poollayer = nn.MaxPool2d(kernel_size=2, stride=2)
        self.features = nn.SequentialCell(OrderedDict([
            ('norm', nn.BatchNorm2d(num_input_features)),
            ('relu', nn.ReLU()),
            ('conv', conv1x1(num_input_features, num_output_features)),
            ('pool', poollayer)
        ]))

    def construct(self, x):
        x = self.features(x)
        return x

+ 构建DenseNet主干网络框架

In [19]:
class Densenet(nn.Cell):
    """
    the densenet architecture
    """
    __constants__ = ['features']

    def __init__(self, growth_rate, block_config, num_init_features=None, bn_size=4, drop_rate=0):
        super(Densenet, self).__init__()

        layers = OrderedDict()
        if num_init_features:
            layers['conv0'] = conv7x7(3, num_init_features, stride=2, padding=3)
            layers['norm0'] = nn.BatchNorm2d(num_init_features)
            layers['relu0'] = nn.ReLU()
            layers['pool0'] = nn.MaxPool2d(kernel_size=3, stride=2, pad_mode='same')
            num_features = num_init_features
        else:
            layers['conv0'] = conv3x3(3, growth_rate*2, stride=1, padding=1)
            layers['norm0'] = nn.BatchNorm2d(growth_rate*2)
            layers['relu0'] = nn.ReLU()
            num_features = growth_rate * 2

        # Each denseblock
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate
            )
            layers['denseblock%d'%(i+1)] = block
            num_features = num_features + num_layers*growth_rate

            if i != len(block_config)-1:
                if num_init_features:
                    trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2,
                                        avgpool=False)
                else:
                    trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2,
                                        avgpool=True)
                layers['transition%d'%(i+1)] = trans
                num_features = num_features // 2

        # Final batch norm
        layers['norm5'] = nn.BatchNorm2d(num_features)
        layers['relu5'] = nn.ReLU()

        self.features = nn.SequentialCell(layers)
        self.out_channels = num_features

    def construct(self, x):
        x = self.features(x)
        return x

    def get_out_channels(self):
        return self.out_channels

    
def _densenet121(**kwargs):
    return Densenet(growth_rate=32, block_config=(6, 12, 24, 16), num_init_features=64, **kwargs)

+ 构建DenseNet121网络

In [20]:
class DenseNet121(nn.Cell):
    """
    the densenet121 architecture
    """
    def __init__(self, num_classes, include_top=True):
        super(DenseNet121, self).__init__()
        self.backbone = _densenet121()
        out_channels = self.backbone.get_out_channels()
        self.include_top = include_top
        if self.include_top:
            self.head = CommonHead(num_classes, out_channels)

        default_recurisive_init(self)
        for _, cell in self.cells_and_names():
            if isinstance(cell, nn.Conv2d):
                cell.weight.set_data(init.initializer(KaimingNormal(a=math.sqrt(5), mode='fan_out',
                                                                    nonlinearity='relu'),
                                                      cell.weight.shape,
                                                      cell.weight.dtype))
            elif isinstance(cell, nn.BatchNorm2d):
                cell.gamma.set_data(init.initializer('ones', cell.gamma.shape))
                cell.beta.set_data(init.initializer('zeros', cell.beta.shape))
            elif isinstance(cell, nn.Dense):
                cell.bias.set_data(init.initializer('zeros', cell.bias.shape))

    def construct(self, x):
        x = self.backbone(x)
        if not self.include_top:
            return x
        x = self.head(x)
        return x

In [21]:
model = DenseNet121(1000, include_top=True)
print(model)

x = Tensor(np.zeros([2, 3, 224, 224]), ms.float32)
out = model(x)

print(out.shape)

DenseNet121<
  (backbone): Densenet<
    (features): SequentialCell<
      (conv0): Conv2d<input_channels=3, output_channels=64, kernel_size=(7, 7), stride=(2, 2), pad_mode=pad, padding=3, dilation=(1, 1), group=1, has_bias=False, weight_init=normal, bias_init=zeros, format=NCHW>
      (norm0): BatchNorm2d<num_features=64, eps=1e-05, momentum=0.09999999999999998, gamma=Parameter (name=backbone.features.norm0.gamma, shape=(64,), dtype=Float32, requires_grad=True), beta=Parameter (name=backbone.features.norm0.beta, shape=(64,), dtype=Float32, requires_grad=True), moving_mean=Parameter (name=backbone.features.norm0.moving_mean, shape=(64,), dtype=Float32, requires_grad=False), moving_variance=Parameter (name=backbone.features.norm0.moving_variance, shape=(64,), dtype=Float32, requires_grad=False)>
      (relu0): ReLU<>
      (pool0): MaxPool2d<kernel_size=3, stride=2, pad_mode=SAME>
      (denseblock1): _DenseBlock<
        (cell_list): CellList<
          (0): _DenseLayer<
            (n

(2, 1000)


In [22]:
import os
import time
import argparse
import datetime
from collections import Counter

import mindspore.nn as nn
from mindspore import Tensor
from mindspore.nn.optim import Momentum
from mindspore.communication.management import init, get_rank, get_group_size
from mindspore.train.callback import ModelCheckpoint
from mindspore.train.callback import CheckpointConfig, Callback
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.model import Model
from mindspore.train.loss_scale_manager import DynamicLossScaleManager, FixedLossScaleManager
from mindspore import context
from mindspore.context import ParallelMode
from mindspore.common import set_seed
from mindspore import save_checkpoint
from mindspore.common import initializer as init
from mindspore.nn.loss.loss import _Loss
from mindspore.ops import operations as P
from mindspore.ops import functional as F
from mindspore.common import dtype as mstype

#from utils.lr_scheduler import MultiStepLR, CosineAnnealingLR
#from utils.crossentropy import CrossEntropy
#from utils.optimizers import get_param_groups

set_seed(1)


### 2.4 自定义回调机制
本案例还利用MindSpore框架，基于Callback基于自定义了回调类ProgressMonitor，为了满足在模型训练时监测训练过程，并在每一个epoch、每一个step以及训练结束后进行loss计算、模型参数保存等。

具体的实现流程就是在自定义的ProgressMonitor类中，通过init方法初始化相关模型保存参数，并重新实现begin、epoch_end、step_end、end等方法，其中在epoch_end方法中通过MindSpore的字典类型变量RunContext.original_args()获取模型训练时记录的相关属性，如cur_epoch_num、cur_step_num，即当前epoch数和step数，以及通过net_outputs得到模型当前损失，在每一个epoc结束后完成loss的输出，step_end方法同理，针对end方法，通过RunContext.original_args()得到train_network模型参数，在训练结束后进行模型的保存。

In [ ]:
class ProgressMonitor(Callback):
    """monitor loss and time"""
    def __init__(self, ckpt_path="checkpoint/", ckpt_name="latest.ckpt"):
        super(ProgressMonitor, self).__init__()
        self.ckpt_path = os.path.join(ckpt_path, ckpt_name)
        
        
    def begin(self, run_context):
        print('start network train...')

    def epoch_begin(self, run_context):
        pass

    def epoch_end(self, run_context, *me_args):
        """process epoch end"""
        cb_params = run_context.original_args()
        me_step = cb_params.cur_step_num

        real_epoch = cb_params.cur_epoch_num
        print('epoch[{}], iter[{}], loss:{},'.format(real_epoch, me_step, cb_params.net_outputs))
            
        
    def step_begin(self, run_context):
        pass

    def step_end(self, run_context, *me_args):
        cb_params = run_context.original_args()
        me_step = cb_params.cur_step_num
        me_batch = cb_params.cur_step_num

        real_epoch = cb_params.cur_epoch_num
        if me_step % 100 == 0:
            print('epoch[{}], iter[{}], loss:{},'.format(real_epoch, me_step, cb_params.net_outputs))

        if me_step % 2400 == 0:
            save_checkpoint(cb_params.train_network, self.ckpt_path)
            print("saving current checkpoint at: {} ".format(self.ckpt_path))
        
    def end(self, run_context):
        cb_params = run_context.original_args()
        save_checkpoint(cb_params.train_network, self.ckpt_path)
        print("saving latest checkpoint at: {} ".format(self.ckpt_path))
        print('end network train...')

### 2.5 模型训练及评估

+ 定义学习率决策

定义了CosineAnnealingLR学习率scheduler决策，以便于在后续的模型训练中使用，主要包含lr, T_max, eta_min等几种参数，其中lr就是设置好的初始学习率，T_max是cosine循环的最大次数，eta_min是在consine循环中最低的学习率，通过修改T_max与eta_min这两个参数可以直接调整训练过程中学习率的变化曲线。

In [23]:
class _WarmUp():
    def __init__(self, warmup_init_lr):
        self.warmup_init_lr = warmup_init_lr

    def get_lr(self):
        # Get learning rate during warmup
        raise NotImplementedError

class _LinearWarmUp(_WarmUp):
    """
    linear warmup function
    """
    def __init__(self, lr, warmup_epochs, steps_per_epoch, warmup_init_lr=0):
        self.base_lr = lr
        self.warmup_init_lr = warmup_init_lr
        self.warmup_steps = int(warmup_epochs * steps_per_epoch)

        super(_LinearWarmUp, self).__init__(warmup_init_lr)

    def get_warmup_steps(self):
        return self.warmup_steps

    def get_lr(self, current_step):
        lr_inc = (float(self.base_lr) - float(self.warmup_init_lr)) / float(self.warmup_steps)
        lr = float(self.warmup_init_lr) + lr_inc * current_step
        return lr

class _LRScheduler():

    def __init__(self, lr, max_epoch, steps_per_epoch):
        self.base_lr = lr
        self.steps_per_epoch = steps_per_epoch
        self.total_steps = int(max_epoch * steps_per_epoch)

    def get_lr(self):
        # Compute learning rate using chainable form of the scheduler
        raise NotImplementedError
        
        
        
class CosineAnnealingLR(_LRScheduler):
    def __init__(self, lr, T_max, steps_per_epoch, max_epoch, warmup_epochs=0, eta_min=0):
        self.T_max = T_max
        self.eta_min = eta_min
        self.warmup = _LinearWarmUp(lr, warmup_epochs, steps_per_epoch)
        super(CosineAnnealingLR, self).__init__(lr, max_epoch, steps_per_epoch)

    def get_lr(self):
        warmup_steps = self.warmup.get_warmup_steps()

        lr_each_step = []
        current_lr = self.base_lr
        for i in range(self.total_steps):
            if i < warmup_steps:
                lr = self.warmup.get_lr(i+1)
            else:
                cur_ep = i // self.steps_per_epoch
                if i % self.steps_per_epoch == 0 and i > 0:
                    current_lr = self.eta_min + \
                                 (self.base_lr - self.eta_min) * (1. + math.cos(math.pi*cur_ep / self.T_max)) / 2

                lr = current_lr

            lr_each_step.append(lr)

        return np.array(lr_each_step).astype(np.float32)

+ 定义交叉熵损失函数

In [ ]:
class CrossEntropy(_Loss):
    """
    loss function CrossEntropy
    """
    def __init__(self, smooth_factor=0., num_classes=1000):
        super(CrossEntropy, self).__init__()
        self.onehot = P.OneHot()
        self.on_value = Tensor(1.0 - smooth_factor, mstype.float32)
        self.off_value = Tensor(1.0 * smooth_factor / (num_classes -1), mstype.float32)
        self.ce = nn.SoftmaxCrossEntropyWithLogits()
        self.mean = P.ReduceMean(False)

    def construct(self, logit, label):
        one_hot_label = self.onehot(label,
                                    F.shape(logit)[1], self.on_value, self.off_value)
        loss = self.ce(logit, one_hot_label)
        loss = self.mean(loss, 0)
        return loss

+ 模型训练

在模型训练时，首先是设置模型训练的epoch次数，再通过2.1节中自定义的classification_dataset方法创建了训练集，其中训练集batch_size大小为4；损失函数使用CrossEntropy，优化器使用Momentum，并设置初始学习率为0.1。回调函数方面使用了自定义的ProgressMonitor来监控训练过程中每个epoch和每个step结束后，损失值Loss的变化情况，并在训练结束后保存当前最优模型。

In [25]:
class BuildTrainNetwork(nn.Cell):
    """build training network"""
    def __init__(self, network, criterion):
        super(BuildTrainNetwork, self).__init__()
        self.network = network
        self.criterion = criterion

    def construct(self, input_data, label):
        output = self.network(input_data)
        loss = self.criterion(output, label)
        return loss

def train():    
    data_dir = "./dataset/train"
    de_dataset = classification_dataset(data_dir, image_size=[84, 84],
                                       per_batch_size=4, max_epoch=20,
                                       rank=0, group_size=1)

    de_dataset.map_model = 4
    epoch = 1
    steps_per_epoch = de_dataset.get_dataset_size()
    ckpt_interval = steps_per_epoch

    network = DenseNet121(100)

    criterion = CrossEntropy(smooth_factor=0.0, num_classes=100)

    lr_scheduler = CosineAnnealingLR(lr=0.1,
                                     T_max=120,
                                     steps_per_epoch=steps_per_epoch,
                                     max_epoch=epoch,
                                     warmup_epochs=0,
                                     eta_min=0)
    lr_schedule = lr_scheduler.get_lr()

    opt = Momentum(params=get_param_groups(network),
                       learning_rate=Tensor(lr_schedule),
                       momentum=0.9,
                       weight_decay=0.0001,
                       loss_scale=1024)

    criterion.add_flags_recursive(fp32=True)


    train_net = BuildTrainNetwork(network, criterion)
    loss_scale_manager = FixedLossScaleManager(1024, drop_overflow_update=False)
    model = Model(train_net, optimizer=opt, metrics=None, loss_scale_manager=loss_scale_manager, amp_level="O3")

    progress_cb = ProgressMonitor(ckpt_path="checkpoint/", ckpt_name="cur.ckpt")
    callbacks = [progress_cb,]

    '''
    outputs_dir = os.path.join("checkpoint/",datetime.datetime.now().strftime('%Y-%m-%d_time_%H_%M_%S'))
    ckpt_max_num = epoch * steps_per_epoch // ckpt_interval
    ckpt_config = CheckpointConfig(save_checkpoint_steps=ckpt_interval,
                                           keep_checkpoint_max=ckpt_max_num)
    ckpt_cb = ModelCheckpoint(config=ckpt_config, directory=outputs_dir)
    callbacks.append(ckpt_cb)
    '''

    model.train(epoch, de_dataset, callbacks=callbacks)
    
train()

[WARNING] ME(95841:8663750144,MainProcess):2022-09-25-17:50:32.873.971 [mindspore/dataset/core/validator_helpers.py:804] 'RandomCropDecodeResize' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'RandomCropDecodeResize' from mindspore.dataset.vision instead.
[WARNING] ME(95841:8663750144,MainProcess):2022-09-25-17:50:32.875.752 [mindspore/dataset/core/validator_helpers.py:804] 'RandomHorizontalFlip' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'RandomHorizontalFlip' from mindspore.dataset.vision instead.
[WARNING] ME(95841:8663750144,MainProcess):2022-09-25-17:50:32.876.563 [mindspore/dataset/core/validator_helpers.py:804] 'RandomColorAdjust' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'RandomColorAdjust' from mindspore.dataset.vision instead.
[WARNING] ME(95841:86637501

start network train...
epoch[1], iter[100], loss:4.6486244,
epoch[1], iter[200], loss:4.7542605,
epoch[1], iter[300], loss:4.5621214,
epoch[1], iter[400], loss:4.945469,
epoch[1], iter[500], loss:4.3907814,
epoch[1], iter[600], loss:4.4915524,
epoch[1], iter[700], loss:4.5935445,
epoch[1], iter[800], loss:4.8271093,
epoch[1], iter[900], loss:4.6720195,
epoch[1], iter[1000], loss:4.9436717,
epoch[1], iter[1100], loss:4.4820595,
epoch[1], iter[1200], loss:4.9001513,
epoch[1], iter[1300], loss:4.753376,
epoch[1], iter[1400], loss:4.750457,
epoch[1], iter[1500], loss:4.9701996,
epoch[1], iter[1600], loss:4.920564,
epoch[1], iter[1700], loss:4.5729656,
epoch[1], iter[1800], loss:4.9123254,
epoch[1], iter[1900], loss:4.500539,
epoch[1], iter[2000], loss:4.9597597,
epoch[1], iter[2100], loss:4.815259,
epoch[1], iter[2200], loss:4.9325304,
epoch[1], iter[2300], loss:5.0505743,
epoch[1], iter[2400], loss:4.757143,
saving current checkpoint at: checkpoint/cur.ckpt 
epoch[1], iter[2500], loss:4.7

### 2.6 模型效果评估

模型效果评估方面首先是通过2.1节中自定义的classification_dataset方法创建了验证集，之后通过mindspore提供的load_checkpoint方法加载模型参数，再将验证集输入模型完成预测，并分别计算预测类别的Top-1和Top-5准确率。

In [15]:
import argparse
import glob

from mindspore.communication.management import init, get_rank, get_group_size, release
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.ops import operations as P
from mindspore.ops import functional as F
from mindspore.common import dtype as mstype
from mindspore.common import initializer as init
from mindspore import Tensor

In [16]:
per_batch_size = 4

def get_top5_acc(top5_arg, gt_class):
    sub_count = 0
    for top5, gt in zip(top5_arg, gt_class):
        if gt in top5:
            sub_count += 1
    return sub_count

def evaluation():
    data_dir = "./dataset/val"
    de_dataset = classification_dataset(data_dir, image_size=[84, 84],
                                       per_batch_size=4, max_epoch=1,
                                       rank=0, group_size=1,mode='eval')
    eval_dataloader = de_dataset.create_tuple_iterator()
    
    network = DenseNet121(100)
    load_checkpoint("checkpoint/cur.ckpt", net=network)
    
    network.add_flags_recursive(fp16=True)
    
    img_tot = 0
    top1_correct = 0
    top5_correct = 0
    network.set_train(False)
    for data, gt_classes in eval_dataloader:
        output = network(Tensor(data, mstype.float32))
        output = output.asnumpy()
        gt_classes = gt_classes.asnumpy()

        top1_output = np.argmax(output, (-1))
        top5_output = np.argsort(output)[:, -5:]

        t1_correct = np.equal(top1_output, gt_classes).sum()
        top1_correct += t1_correct
        top5_correct += get_top5_acc(top5_output, gt_classes)
        img_tot += per_batch_size
        
    results = [[top1_correct], [top5_correct], [img_tot]]
    results = np.array(results)
    
    print('results={}'.format(results))
    
    top1_correct = results[0, 0]
    top5_correct = results[1, 0]
    img_tot = results[2, 0]
    acc1 = 100.0 * top1_correct / img_tot
    acc5 = 100.0 * top5_correct / img_tot
    print('top1_correct={}, tot={}, acc={:.2f}%'.format(top1_correct,img_tot,acc1))
    print('top5_correct={}, tot={}, acc={:.2f}%'.format(top5_correct,img_tot,acc5))
    
    
evaluation()

[WARNING] ME(13820:9872,MainProcess):2022-09-22-19:22:21.823.623 [mindspore\train\serialization.py:667] For 'load_param_into_net', remove parameter network.backbone.features.conv0.weight's prefix name: network., continue to load it to net parameter backbone.features.conv0.weight.


results=[[  237]
 [ 1037]
 [12000]]
top1_correct=237, tot=12000, acc=1.98%
top5_correct=1037, tot=12000, acc=8.64%


### 2.7 模型预测

由于在2.1中使用ImageFolderDataset创建数据集时，会将100个分类的每个分类编号(如"n01532829")转换为0-100的索引，模型针对某张图片得到的预测结果也会是0-100中的某个值，但在模型预测时，不能只预测出0-100的分类索引，还要确定具体的分类名称，所以就需要从目前的0-100的分类索引转换为具体的分类名称

ImageNet提供了存储分类编号与分类名称的json文件(如分类编号"n01532829"对应分类名称"cat")，所以就可以通过pandas读取该json文件，得到分类编号与分类名称的对应关系，之后用字典index_dict存储分类索引与分类编号的对应关系，由此分类索引就可以与分类名称就可以建立起对应关系，最后通过定义class_index_map完成分类索引到分类名称的转换。

具体实现流程就是通过classification_dataset创建测试集，并加载保存好的模型参数完成每一个batch的预测，得到对应的分类索引，并通过class_index_map来根据分类索引获取最终的分类名称。



In [203]:
import pandas as pd

# 根据分类索引列表获取分类名称
def class_index_map(df, predict_output, index_dict):
    result_list = []
    for i in predict_output:
        temp_df = df.loc[:,df.loc[0,:] == index_dict[i]] # 获取分类编号对应的df所有行，index_dict[i]为分类编号，如“n012345”
        class_name = str(temp_df.loc[1,:].values) # 获取分类编号对应的分类名称
        result_list.append(class_name)
        
    return result_list

def prediction(classes_df, index_dict):
    data_dir = "./dataset/test"
    de_dataset = classification_dataset(data_dir, image_size=[84, 84],
                                       per_batch_size=4, max_epoch=1,
                                       rank=0, group_size=1,mode='test')
    eval_dataloader = de_dataset.create_tuple_iterator()
    
    network = DenseNet121(100)
    load_checkpoint("checkpoint/cur.ckpt", net=network)
    
    network.add_flags_recursive(fp16=True)
    
    network.set_train(False)
    i = 0
    img_tot = 0
    for data, gt_classes in eval_dataloader:
        output = network(Tensor(data, mstype.float32))
        output = output.asnumpy()
        gt_classes = gt_classes.asnumpy()

        top1_output = np.argmax(output, (-1))
        top1_output = class_index_map(classes_df, top1_output, index_dict)
        gt_classes = class_index_map(classes_df, gt_classes, index_dict)
        print("predict classification: \n{}".format(top1_output))
        print("real classification: \n{}".format(gt_classes))
        print("--------------------------No.{} batch of test dataset----------------------------".format(i+1))
        i += 1


# 存储分类编号和分类名称，如"n012345==>Cat"
classes_df = pd.read_json('class_dict.json')
# 存储分类索引和分类编号，如“0==>n012345”
index_dict = {}
data_dir = "dataset/train"
index_num = os.listdir(data_dir)
index_num.sort()
i = 0
for index in index_num:
    index_dict[i] = index
    i += 1        
               
prediction(classes_df, index_dict)

[WARNING] ME(9133:8606791168,MainProcess):2022-09-15-17:14:53.701.172 [mindspore/dataset/core/validator_helpers.py:804] 'Decode' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Decode' from mindspore.dataset.vision instead.
[WARNING] ME(9133:8606791168,MainProcess):2022-09-15-17:14:53.701.742 [mindspore/dataset/core/validator_helpers.py:804] 'Resize' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Resize' from mindspore.dataset.vision instead.
[WARNING] ME(9133:8606791168,MainProcess):2022-09-15-17:14:53.702.034 [mindspore/dataset/core/validator_helpers.py:804] 'CenterCrop' from mindspore.dataset.vision.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'CenterCrop' from mindspore.dataset.vision instead.
[WARNING] ME(9133:8606791168,MainProcess):2022-09-15-17:14:53.702.277 [mindspore/dataset/core/validator_

predict classification: 
["['tank']", "['catamaran']", "['catamaran']", "['photocopier']"]
real classification: 
["['robin']", "['catamaran']", "['jellyfish']", "['street_sign']"]
--------------------------No.1 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['mixing_bowl']", "['white_wolf']", "['carton']", "['robin']"]
--------------------------No.2 batch of test dataset----------------------------
predict classification: 
["['catamaran']", "['tank']", "['catamaran']", "['triceratops']"]
real classification: 
["['aircraft_carrier']", "['missile']", "['street_sign']", "['meerkat']"]
--------------------------No.3 batch of test dataset----------------------------
predict classification: 
["['catamaran']", "['tank']", "['catamaran']", "['triceratops']"]
real classification: 
["['solar_dish']", "['clog']", "['chime']", "['white_wolf']"]
--------------------------No.4 batch of test dataset--

predict classification: 
["['ear']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['ladybug']", "['carton']", "['French_bulldog']", "['tile_roof']"]
--------------------------No.33 batch of test dataset----------------------------
predict classification: 
["['tank']", "['catamaran']", "['tank']", "['tank']"]
real classification: 
["['oboe']", "['dishrag']", "['unicycle']", "['Newfoundland']"]
--------------------------No.34 batch of test dataset----------------------------
predict classification: 
["['triceratops']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['meerkat']", "['prayer_rug']", "['vase']", "['iPod']"]
--------------------------No.35 batch of test dataset----------------------------
predict classification: 
["['photocopier']", "['tank']", "['tank']", "['ear']"]
real classification: 
["['vase']", "['robin']", "['hair_slide']", "['boxer']"]
--------------------------No.36 batch of test dataset----------------------------
predict classification:

predict classification: 
["['tank']", "['tank']", "['triceratops']", "['catamaran']"]
real classification: 
["['tank']", "['scoreboard']", "['worm_fence']", "['missile']"]
--------------------------No.67 batch of test dataset----------------------------
predict classification: 
["['consomme']", "['tank']", "['tank']", "['ear']"]
real classification: 
["['golden_retriever']", "['African_hunting_dog']", "['carousel']", "['trifle']"]
--------------------------No.68 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['photocopier']", "['tank']"]
real classification: 
["['clog']", "['catamaran']", "['malamute']", "['bookshop']"]
--------------------------No.69 batch of test dataset----------------------------
predict classification: 
["['triceratops']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['bolete']", "['organ']", "['black-footed_ferret']", "['ant']"]
--------------------------No.70 batch of test dataset--------------

predict classification: 
["['photocopier']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['rock_beauty']", "['triceratops']", "['cannon']", "['Newfoundland']"]
--------------------------No.101 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['goose']", "['white_wolf']", "['parallel_bars']", "['vase']"]
--------------------------No.102 batch of test dataset----------------------------
predict classification: 
["['photocopier']", "['catamaran']", "['tank']", "['tank']"]
real classification: 
["['hourglass']", "['coral_reef']", "['file']", "['frying_pan']"]
--------------------------No.103 batch of test dataset----------------------------
predict classification: 
["['triceratops']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['rock_beauty']", "['cocktail_shaker']", "['unicycle']", "['lipstick']"]
--------------------------No.104 batch of test dataset--------

...

predict classification: 
["['tank']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['miniature_poodle']", "['aircraft_carrier']", "['bookshop']", "['upright']"]
--------------------------No.2850 batch of test dataset----------------------------
predict classification: 
["['photocopier']", "['tank']", "['catamaran']", "['tank']"]
real classification: 
["['white_wolf']", "['yawl']", "['miniskirt']", "['boxer']"]
--------------------------No.2851 batch of test dataset----------------------------
predict classification: 
["['ear']", "['green_mamba']", "['catamaran']", "['catamaran']"]
real classification: 
["['carton']", "['hotdog']", "['parallel_bars']", "['tile_roof']"]
--------------------------No.2852 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['photocopier']"]
real classification: 
["['bolete']", "['vase']", "['reel']", "['cannon']"]
--------------------------No.2853 batch of test dataset-------------

predict classification: 
["['tank']", "['photocopier']", "['catamaran']", "['tank']"]
real classification: 
["['miniskirt']", "['hourglass']", "['catamaran']", "['house_finch']"]
--------------------------No.2884 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['Arctic_fox']", "['poncho']", "['solar_dish']", "['fire_screen']"]
--------------------------No.2885 batch of test dataset----------------------------
predict classification: 
["['tank']", "['catamaran']", "['tank']", "['tank']"]
real classification: 
["['trifle']", "['yawl']", "['Tibetan_mastiff']", "['dalmatian']"]
--------------------------No.2886 batch of test dataset----------------------------
predict classification: 
["['catamaran']", "['green_mamba']", "['ear']", "['tank']"]
real classification: 
["['dugong']", "['Saluki']", "['pencil_box']", "['organ']"]
--------------------------No.2887 batch of test dataset-------------

predict classification: 
["['tank']", "['catamaran']", "['tank']", "['tank']"]
real classification: 
["['ladybug']", "['solar_dish']", "['carousel']", "['combination_lock']"]
--------------------------No.2918 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['photocopier']"]
real classification: 
["['beer_bottle']", "['slot']", "['trifle']", "['vase']"]
--------------------------No.2919 batch of test dataset----------------------------
predict classification: 
["['catamaran']", "['tank']", "['triceratops']", "['tank']"]
real classification: 
["['yawl']", "['upright']", "['oboe']", "['iPod']"]
--------------------------No.2920 batch of test dataset----------------------------
predict classification: 
["['catamaran']", "['tank']", "['catamaran']", "['tank']"]
real classification: 
["['Walker_hound']", "['upright']", "['chime']", "['Walker_hound']"]
--------------------------No.2921 batch of test dataset----------------------

predict classification: 
["['catamaran']", "['tank']", "['consomme']", "['ear']"]
real classification: 
["['goose']", "['ladybug']", "['orange']", "['toucan']"]
--------------------------No.2952 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['rhinoceros_beetle']", "['stage']", "['bolete']", "['harvestman']"]
--------------------------No.2953 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['ear']", "['tank']"]
real classification: 
["['scoreboard']", "['ladybug']", "['chime']", "['Walker_hound']"]
--------------------------No.2954 batch of test dataset----------------------------
predict classification: 
["['tank']", "['photocopier']", "['tank']", "['tank']"]
real classification: 
["['cannon']", "['mixing_bowl']", "['cocktail_shaker']", "['harvestman']"]
--------------------------No.2955 batch of test dataset-------------------------

predict classification: 
["['tank']", "['tank']", "['tank']", "['photocopier']"]
real classification: 
["['scoreboard']", "['crate']", "['meerkat']", "['harvestman']"]
--------------------------No.2986 batch of test dataset----------------------------
predict classification: 
["['tank']", "['photocopier']", "['photocopier']", "['triceratops']"]
real classification: 
["['white_wolf']", "['iPod']", "['malamute']", "['theater_curtain']"]
--------------------------No.2987 batch of test dataset----------------------------
predict classification: 
["['tank']", "['tank']", "['tank']", "['tank']"]
real classification: 
["['combination_lock']", "['bookshop']", "['house_finch']", "['prayer_rug']"]
--------------------------No.2988 batch of test dataset----------------------------
predict classification: 
["['tank']", "['photocopier']", "['tank']", "['catamaran']"]
real classification: 
["['miniature_poodle']", "['snorkel']", "['upright']", "['coral_reef']"]
--------------------------No.2989 batc

## 3 总结

本案例基于MindSpore框架针对mini-ImageNet数据集，首先在对原始数据集进行预处理后完成了数据读取与数据集创建，之后通过框架构建了Densenet121模型并自定义了回调函数，并在模型训练后，进行了分类索引与数据集提供的分类编号、分类名称的转换，顺利利用模型完成了评估和预测。通过此案例进一步加深了对Densenet模型结构和特性的理解，并通过使用MindSpore框架高效地完成了整个案例实现流程，加深了对框架提供的API的理解与掌握。